In [1]:
from LSTM_BigBirdA2C_GumbelSoftmax import *
from dataset import make_data_generator
import json
from tqdm import tqdm_notebook as tqdm

In [2]:
device = torch.device('cuda')

In [3]:
folder = 'data/food/'
data_name = folder+'data.json'
# validation_name = folder+'valid_seq.json'
# testdata_name = folder+'testdata_seq.json'
vocab_name = folder+'vocab.json'

In [4]:
vocab = json.load(open(vocab_name, 'r'))
VOC_SIZE = len(vocab)
INPUT_MAX = 100
SUMM_MAX = 20
UNK = "[UNK]"
BOS = "[CLS]"
EOS = "[SEP]"
PAD = "[PAD]"

In [5]:
batch_size = 16

training_set, training_generator = make_data_generator(\
data_name, INPUT_MAX, SUMM_MAX, vocab[PAD], batch_size, cutoff=None, shuffle=True, num_workers=4)

# validation_set, validation_generator = make_data_generator(\
# validation_name, INPUT_MAX, OUTPUT_MAX, vocab[PAD], batch_size, cutoff=validation_size, shuffle=False, num_workers=4)

def data_gen_train():
    for src, label, tgt in training_generator:
        src = src.to(device)
        label = (label).long().to(device)
        tgt = tgt.to(device)
        b = Batch(src, tgt, vocab[PAD])
        b.label = label
        yield b

loading json
load json done.


In [6]:
import math
total_train = int(math.ceil(training_set.size / batch_size))
# total_valid = int(math.ceil(validation_set.size / batch_size))
# print(total_train, total_valid)

In [7]:
def init_param(model):
    for p in model.parameters():
        if p.dim() > 1:
            nn.init.xavier_uniform_(p)

def make_big_bird(vocab, N=6, 
               d_model=512, d_ff=2048, h=8, dropout=0.1, emb_share=False, bert_share=False):
    "Helper: Construct a model from hyperparameters."
    
    vocab_sz = len(vocab)
    
    c = copy.deepcopy
    attn = MultiHeadedAttention(h, d_model)
    ff = PositionwiseFeedForward(d_model, d_ff, dropout)
    position = PositionalEncoding(d_model, dropout)
    
    src_emb = nn.Sequential(Embeddings(d_model, vocab_sz), c(position))
    if emb_share:        
        tgt_emb = src_emb
        bert_class_emb = src_emb
        bert_discr_emb = src_emb
    else:
        tgt_emb = nn.Sequential(Embeddings(d_model, vocab_sz), c(position))
        bert_class_emb = nn.Sequential(Embeddings(d_model, vocab_sz), c(position))
        bert_discr_emb = nn.Sequential(Embeddings(d_model, vocab_sz), c(position))
    
    
    bert_class = BERT(
        Encoder(EncoderLayer(d_model, c(attn), c(ff), dropout), N),
        bert_class_emb,
        vocab[PAD]
    )
    
    if bert_share:
        bert_discr = bert_class
    else:
        bert_discr = BERT(
            Encoder(EncoderLayer(d_model, c(attn), c(ff), dropout), N),
            bert_discr_emb,
            vocab[PAD]
        )
        
    translator = LSTM_Gumbel_Encoder_Decoder(
        hidden_dim=d_model, 
        emb_dim=d_model, 
        input_len=INPUT_MAX, 
        output_len=SUMM_MAX, 
        voc_size=vocab_sz, 
        critic_net=CriticNet(2*d_model),
        device=device,
        eps=1e-8
    )
    
    reconstructor = LSTM_Normal_Encoder_Decoder(
        hidden_dim=d_model, 
        emb_dim=d_model, 
        input_len=SUMM_MAX, 
        output_len=INPUT_MAX, 
        voc_size=vocab_sz, 
        pad_index=vocab[PAD],
        eps=1e-8
    )
#     translator = Translator(
#         Encoder(EncoderLayer(d_model, c(attn), c(ff), dropout), N),
#         Decoder(DecoderLayer(d_model, c(attn), c(attn), 
#                              c(ff), dropout), N),
#         src_emb,
#         tgt_emb,
#         Generator(d_model, vocab_sz, device),
#         CriticNet(d_model)
#         )
    
#     classifier = Classifier(
#         bert_class,
#         out_class = 5
#         # criterion = BCE
#     )
#     reconstructor = Reconstructor(
#         Encoder(EncoderLayer(d_model, c(attn), c(ff), dropout), N),
#         Decoder(DecoderLayer(d_model, c(attn), c(attn), 
#                              c(ff), dropout), N),
#         src_emb,
#         tgt_emb,
#         Generator(d_model, vocab_sz, device),
#         vocab[PAD]
#     )   
    discriminator = Discriminator(
        Encoder(EncoderLayer(d_model, c(attn), c(ff), dropout), N),
        d_model,
        len(vocab),
        vocab[PAD]
    )


    # This was important from their code. 
    # Initialize parameters with Glorot / fan_avg.
    for m in [translator, reconstructor, discriminator]:
        init_param(m)
        
#     if(str(device) == 'cpu'):
#         savedmodel = torch.load("pretrained/Translator4", map_location=lambda storage, location: storage)
#     else:
#         savedmodel = torch.load("pretrained/Translator4")
#     translator.load_state_dict(savedmodel['model'], strict=False)
#     if(str(device) == 'cpu'):
#         savedmodel = torch.load("pretrained/Translator4", map_location=lambda storage, location: storage)
#     else:
#         savedmodel = torch.load("pretrained/Translator4")
#     reconstructor.load_state_dict(savedmodel['model'])#, strict=False)
            
    # creation of big bird
    model = BigBird(
        translator, discriminator, reconstructor , 
        vocab, gamma=0.99, clip_value=0.5, #for WGAN, useless if WGAN-GP is used 
        lr_G = 5e-5,
        lr_D = 1e-4,
        lr_R = 1e-4,
        LAMBDA = 10, # Gradient penalty lambda hyperparameter
        RL_scale = 10,
        device = device
    )

    return model


In [8]:
model = make_big_bird(vocab, N=4, d_model=256, d_ff=256, h=4, dropout=0.1, emb_share=True, bert_share=True)
model.load("Nest/NewbornBirdA2C_LSTM_GumbelSoftmax")

load Bird from Nest/NewbornBirdA2C_LSTM_GumbelSoftmax


In [ ]:
vocab_inv = {a:b for b, a in vocab.items()}
def convert_ids_to_tokens(ids):
    return [vocab_inv[i] for i in ids]

In [ ]:
trange = tqdm(enumerate(data_gen_train()), total=total_train)
all_acc = [];
all_reward = [];
ct = 0
for i, batch in trange:
    acc, reward = model.eval_iter(batch.src, batch.src_mask, SUMM_MAX, batch.trg, ct)
    all_acc.append(acc)
    all_reward.append(reward)
    ct += 1

origin:
['[CLS]', 'after', 'trying', 'gu', 'gel', '(', 'which', 'made', 'me', 'gag', ')', ',', 'i', 'decided', 'to', 'try', 'these', 'beans', '.', 'the', 'water', '##mel', '##on', 'flavor', 'was', 'awesome', ',', 'no', 'fake', 'taste', 'like', 'the', 'gel', '##s', 'that', 'i', "'", 've', 'tried', '.', 'they', 'go', 'down', 'as', 'easy', 'as', 'regular', 'jelly', '##be', '##ans', '.', 'do', 'drink', 'with', 'water', 'though', 'to', 'prevent', 'stomach', 'upset', '(', 'it', 'tells', 'you', 'that', 'on', 'the', 'box', ')', '.', 'i', 'haven', "'", 't', 'had', 'any', 'problems', 'with', 'them', 'at', 'all', ',', 'as', 'long', 'as', 'i', 'drink', 'water', '.', 'they', 'give', 'me', 'the', 'most', 'energy', 'of', 'any', 'i', "'", 've']
summary:
['[CLS]', 'some', 'pouch', '[CLS]', '[CLS]', 'chef', '##inger', 'great', '[PAD]', 'the', 'place', '[PAD]', '[PAD]', 'make', '[PAD]', '[PAD]', '[PAD]', 'the', '[PAD]', '.']
real summary:
['[CLS]', 'these', 'taste', 'awesome', ',', 'and', 'do', 'the', 'j

origin:
['[CLS]', 'this', 'coffee', 'actually', 'tastes', 'pretty', 'good', 'for', 'a', 'darker', 'roast', 'however', 'the', 'number', 'of', 'defective', 'k', '-', 'cups', 'with', 'seal', 'problems', 'can', 'be', 'significant', '.', 'the', 'issue', 'is', 'that', 'the', 'seal', 'covering', 'the', 'kc', '##up', ',', 'which', 'gets', 'pun', '##cture', '##d', 'to', 'enable', 'the', 'drip', 'through', ',', 'seems', 'to', 'continually', 'burst', 'open', 'during', 'brewing', ',', 'resulting', 'in', 'coffee', 'grounds', 'in', 'your', 'mug', 'and', 'quite', 'a', 'mess', 'of', 'the', 'machine', '.', 'my', 'wife', 'and', 'i', 'have', 'started', 'calling', 'these', '"', 'burst', '##ers', '"', 'as', 'it', 'happens', 'quite', 'frequently', 'with', 'these', 'em', '##eri', '##l', "'", 's', 'kc', '##ups', '.', 'you', 'can']
summary:
['[CLS]', '##ened', '*', '[SEP]', '[CLS]', '##nos', 'food', '##cken', '[PAD]', 'green', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', 'is', '[PAD]', '[PAD]', 'out', ';']
rea

origin:
['[CLS]', 'recently', 'discovered', 'these', '7', 'oz', 'cans', 'of', 'salsa', 'by', 'em', '##bas', '##a', '.', 'i', "'", 've', 'liked', 'every', 'one', 'i', "'", 've', 'tried', ',', 'and', 'this', 'salsa', 'verde', 'is', 'no', 'exception', '.', '<', 'br', '/', '>', '<', 'br', '/', '>', 'likes', ':', '<', 'br', '/', '>', '<', 'br', '/', '>', '-', 'spicy', '.', 'medium', 'hot', '.', '<', 'br', '/', '>', '-', 'delicious', '.', 'great', 'snack', 'with', 'chips', '.', 'surprisingly', 'good', 'coming', 'out', 'of', 'a', 'can', '.', '<', 'br', '/', '>', '-', 'healthy', ':', '35', 'cal', '##ories', 'per', 'can', ',', 'zero', 'fat', '.', '<', 'br', '/', '>', '<', 'br', '/']
summary:
['[CLS]', '##ei', '##uc', '[SEP]', '[CLS]', 'chicken', '##ctic', 'j', '[PAD]', '(', '!', '[PAD]', '?', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[SEP]', '[PAD]', '[SEP]']
real summary:
['[CLS]', 'excellent', ',', 'spicy', 'tom', '##ati', '##llo', 'salsa', ',', 'for', 'a', 'can', '.', '[SEP]', '[PAD]', '[PAD]', '

origin:
['[CLS]', 'this', 'is', 'not', 'as', 'good', 'as', 'the', 'wal', '##mart', 'private', 'label', 'or', 'mei', '##jer', 'or', 'target', '.', 'it', 'doesn', "'", 't', 'melt', 'in', 'the', 'mouth', 'as', 'quickly', 'and', 'it', 'isn', "'", 't', 'as', 'flavor', '##ful', '.', 'also', 'it', 'has', 'lac', '##tic', 'acid', 'este', '##rs', 'of', 'mono', 'and', 'dig', '##ly', '##cer', '##ides', '.', 'whatever', 'that', 'is', ',', 'i', 'don', "'", 't', 'think', 'my', 'child', 'needs', 'it', '.', '<', 'br', '/', '>', '<', 'br', '/', '>', 'honest', 'mom', '[SEP]']
summary:
['[CLS]', '##ories', 'brewers', 'though', '[CLS]', 'poor', '##er', 'fighter', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', 'more', '[PAD]', 'my', '[PAD]', '[PAD]', '[SEP]', 'or']
real summary:
['[CLS]', 'not', 'as', 'good', 'as', 'store', 'brands', '[SEP]', '[PAD]']
reconsturct out:
['[CLS]', '[CLS]', '[CLS]', '[CLS]', '[CLS]', '[CLS]', '[CLS]', '[CLS]', '[CLS]', '[CLS]', '[CLS]', '[CLS]', '[CLS]', '[CLS]', '[CLS]', '[CLS]'

origin:
['[CLS]', 'we', 'love', 'flavor', '##ed', 'coffee', '##s', 'but', 'the', 'price', 'is', 'just', 'to', 'high', 'so', 'i', 'bought', 'orange', 'flavor', ',', 'hazel', '##nut', 'flavor', ',', 'cherry', 'flavor', ',', 'cara', '##mel', 'flavor', ',', 'and', 'chocolate', 'flavor', 'powers', ',', 'to', 'make', 'my', 'own', 'and', 'they', 'work', 'very', 'well', '.', 'we', 'can', 'flavor', 'by', 'the', 'cup', 'or', 'the', 'pot', 'and', 'we', 'are', 'very', 'happy', 'with', 'them', '.', '<', 'br', '/', '>', '<', 'br', '/', '>', 'with', 'some', 'experimentation', 'we', 'fond', 'all', 'but', 'the', 'chocolate', 'dissolve', 'in', 'cold', '.', 'so', 'we', 'can', 'and', 'do', 'use', 'it', 'in', 'ice', 'coffee', ',', 'and', 'we', 'can', 'now', 'make']
summary:
['[CLS]', '[SEP]', '*', 'their', '[CLS]', 'nail', 'great', 'great', '##h', '##ei', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '!', '[PAD]', 'is', 'the', '[PAD]', '?']
real summary:
['[CLS]', 'oh', 'so', 'vi', '##rita', '##l', 'flavor', '##ings

...

origin:
['[CLS]', 'after', 'reading', 'the', 'reviews', 'i', 'was', 'it', '##ching', 'to', 'get', 'my', 'hands', 'on', 'these', 'as', 'the', 'ones', 'i', 'had', '<', 'br', '/', '>', 'were', 'a', 'couple', 'of', 'years', 'old', 'but', 'kept', 'sealed', ',', 'so', 'they', 'were', 'ok', '.', '<', 'br', '/', '>', 'if', 'you', 'keep', 'pi', '##sta', '##chio', '##s', 'sealed', 'they', 'last', 'a', 'very', ',', 'very', 'long', 'time', '.', '<', 'br', '/', '>', 'the', 'new', 'pi', '##sta', '##chio', '##s', 'did', 'not', 'taste', 'much', 'better', 'than', 'my', 'two', 'year', 'old', 'ones', '.', '<', 'br', '/', '>', 'nu', '##ff', 'said', '[SEP]']
summary:
['[CLS]', 'add', 'free', '[SEP]', 'trap', '##ten', 'very', 'kate', 'of', 'syrup', '?', '[PAD]', '[PAD]', '[PAD]', '!', 'free', '[PAD]', 'is', 'is', '.']
real summary:
['[CLS]', 'ok', ',', 'but', 'not', 'great', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]']
reconsturct out:
['[CLS]', '[CLS]', '[CLS]', '[CLS]', '[CLS]',

origin:
['[CLS]', 'i', 'don', "'", 't', 'have', 'a', 'dog', ',', 'but', 'my', 'neighbor', 'does', 'and', 'her', 'po', '##och', 'loved', 'these', 'treats', '.', 'her', 'dog', 'is', 'somewhat', 'cho', '##os', '##y', 'with', 'food', 'and', 'treats', ',', 'so', 'this', 'one', 'was', 'real', 'winner', '.', '[SEP]']
summary:
['[CLS]', '##ei', 'from', '$', '[CLS]', 'extract', 'hall', 'these', 'cups', 'best', 'very', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[SEP]', 'some', 'free']
real summary:
['[CLS]', 'happy', 'dog', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]']
reconsturct out:
['[CLS]', '[CLS]', '[CLS]', '[CLS]', '[CLS]', '[CLS]', '[CLS]', '[CLS]', '[CLS]', '[CLS]', '[CLS]', '[CLS]', '[CLS]', '[CLS]', '[CLS]', '[CLS]', '[CLS]', '[CLS]', '[CLS]', '[CLS]', '[CLS]', '[CLS]', '[CLS]', '[CLS]', '[CLS]', '[CLS]', '[CLS]', '[CLS]', '[CLS]', '[CLS]', '[CLS]', '[CLS]', '[CLS]', '[CLS]', '[CLS]', '[CLS]', '[CLS]', '[CLS]', '[CLS]', '[CLS]', '[CLS]']

origin:
['[CLS]',

origin:
['[CLS]', 'i', 'actually', 'google', '##d', 'this', ',', 'so', 'bear', 'with', 'me', '.', 'this', 'is', 'ethiopian', 'yi', '##rga', '##che', '##ffe', ',', 'not', 'ethiopian', 'ha', '##rra', '##r', ',', 'which', 'apparently', 'is', 'different', '.', '"', 'yi', '##rga', '##che', '##ffe', 'is', 'known', 'for', 'its', 'orange', 'flower', 'scent', '.', '"', 'having', 'never', 'smelled', 'an', 'orange', 'flower', ',', 'i', 'couldn', "'", 't', 'tell', 'you', '.', 'what', 'i', 'can', 'tell', 'you', 'is', 'that', 'is', 'ethiopian', 'that', 'isn', "'", 't', 'burned', 'like', 'that', 'well', '-', 'known', 'chain', "'", 's', 'version', '.', 'it', 'also', 'has', 'cute', 'life', '-', 'af', '##firm', '##ing', 'things', 'on', 'the', 'box', 'like', ',', '"', 'profits']
summary:
['[CLS]', 'some', '##ories', '[CLS]', 'pouch', 'history', '##s', 'rec', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '!']
real summary:
['[CLS]', 'good', 'ethiopian',

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



origin:
['[CLS]', 'trying', 'to', 'eat', 'more', 'un', '##pro', '##ces', '##sed', 'and', 'natural', 'foods', ',', 'i', 'had', 'been', 'interested', 'in', 'coconut', 'mann', '##a', 'although', 'i', 'wasn', "'", 't', 'quite', 'sure', 'what', 'to', 'expect', '.', 'it', "'", 's', 'basically', 'a', 'coconut', 'cream', 'made', 'from', 'organic', 'dried', 'coconut', 'meat', 'that', 'is', 'spread', '##able', 'like', 'butter', '(', 'a', 'little', 'more', 'grit', '##ty', 'than', 'butter', ',', 'though', ')', '.', '<', 'br', '/', '>', '<', 'br', '/', '>', 'when', 'i', 'received', 'it', ',', 'i', 'opened', 'the', 'container', '-', 'the', 'wonderful', 'smell', 'of', 'coconut', 'wa', '##fted', 'up', ',', 'but', 'it', 'had', 'clearly', 'melted', 'and', 'res', '##oli', '##di', '##fied']
summary:
['[CLS]', 'is', 'free', '[CLS]', '[CLS]', '##red', 'scripture', 'there', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '!', '[PAD]', '[PAD]', 'is', '[PAD]', '[PAD]', '[SEP]', '!']
real summary:
['[CLS]', 'love', 'the', 

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot(range(len(all_reward)), all_reward)
print("[eval] epoch reward avg:",sum(all_reward)/len(all_reward))

In [ ]:
plt.plot(range(len(all_acc)), all_acc)
print("[eval] epoch acc avg:",sum(all_acc)/len(all_acc))